In [289]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm

**Geometric Brownian Motion (GBM)**

In [45]:
class GeometricBrownianMotion:
    """
    input: 
        * nSteps : 步數, the number of samples per path
        * nPaths : 路徑數, the number of paths to generate
        * mu : 平均值, the drift
        * S0 : 期初股價, the initial stock price of the sample path
        * sigma : 標準差, the standard deviation
        * T : 時間
    output:
        * Rt : array
        * lnSt : array
        * St : 每期股價, array
        * df : 每期股價, dataframe
    """
    def __init__(self, nSteps=100, nPaths=10000, mu=0.05, S0=100, sigma=0.3, T=1):
        dt = T/nSteps    # 計算時間步長
        Rt = (mu - sigma ** 2 / 2) * dt + sigma * np.random.normal(0, np.sqrt(dt), size=(nSteps, nPaths))    # 計算每一期的增量/漲跌幅
        St = np.exp( Rt )    # 計算每個時間步的股價
        St = np.vstack([np.ones(nPaths), St])    # 將初始股價添加到St的開始處
        St = S0 * St.cumprod(axis=0)    # 計算股價的累積乘積
        time = np.linspace(0, T, nSteps+1)    # 生成時間值
        
        self.Rt = np.vstack([np.zeros(nPaths), Rt])
        self.lnSt = (1+self.Rt).cumprod(axis=0)
        self.St = St
        self.df = pd.DataFrame(St, index=time)

In [76]:
class call:
    def __init__(self, S0, K, r, q, sigma, T):
        d1 = ( np.log(S0/K) + ( r-q + sigma**2 / 2 ) * T ) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        ert = np.exp(-r * T)
        eqt = np.exp(-q * T)
        self.price = S0 * np.exp(-q * T) * norm.cdf(d1) - K * ert * norm.cdf(d2)
        self.delta = norm.cdf(d1) * eqt
        self.gamma = norm.pdf(d1) / (S0 * sigma * np.sqrt(T)) * eqt
        self.vega = S0 * norm.pdf(d1) * np.sqrt(T) * eqt
        self.theta = (-S0 * norm.pdf(d1) * sigma * ert) / (2 * np.sqrt(T)) - r * K * ert * norm.cdf(d2) \
                + q * S0 *norm.cdf(d1) * eqt
        self.rho = 0 
        self.greek = np.array([self.delta,self.gamma,self.vega,self.theta])
        self.S0 = S0
        self.K = K
        self.r = r
        self.q = q
        self.sigma = sigma
        self.T = T
call(S0=100, K=80, r=0.1, q=0.03, sigma=0.3, T=1).price
class put:
    def __init__(self, S0, K, r, q, sigma, T):
        d1 = ( np.log(S0/K) + ( r-q + sigma**2 / 2 ) * T ) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        ert = np.exp(-r * T)
        eqt = np.exp(-q * T)
        self.price =  K * ert * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)
        self.delta = norm.cdf(d1) * eqt - 1
        self.gamma = norm.pdf(d1) / (S0 * sigma * np.sqrt(T)) * eqt
        self.vega = S0 * norm.pdf(d1) * np.sqrt(T) * eqt
        self.theta = (-S0 * norm.pdf(d1) * sigma * ert) / (2 * np.sqrt(T)) + r * K * ert * norm.cdf(-d2) \
                - q * S0 *norm.cdf(-d1) * eqt
        self.rho = 0 
        self.greek = np.array([self.delta,self.gamma,self.vega,self.theta])
        self.S0 = S0
        self.K = K
        self.r = r
        self.q = q
        self.sigma = sigma
        self.T = T

# **1. Holder-Extendible Call**
Consider an extendible call with initial time to maturity six months, extendible for an additional three months. The stock price is 100, the initial strike price is 100, the extended strike price is 105, the riskfree interest rate is 8% per year, the volatility is 25% per year, and the extension fee is 1. (9.4029)

In [288]:
S0 = 100
K1 = 100
K2 = 105
r = 0.08
q = 0
sigma = 0.25
extension_fee = 1 #展期費
t1 = 6/12
t2 = 3/12
nSteps = 900
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, t1+t2) # GBM產生9個月的樣本路徑
callprice = call(gbm.df.loc[t1], K2, r, q, sigma, t2).price # 用BS公式計算在t=6個月時，每個路徑當下的選擇權價值(3m到期，K2=105)

payoff1 = np.maximum(gbm.df.loc[t1] - K1, 0)      # 計算假設所有路徑都在t=6時，用K1=100履約的payoff
payoff2 = np.maximum(gbm.df.loc[t1+t2] - K2, 0)   # 計算假設所有路徑都在t=9時，用K2=105履約的payoff

# 在t=6時，若選擇權價值(3m到期，K2=105)-展期成本($1)大於立即履約的payoff1，則展期
extFlag = ( callprice-1 > payoff1 ) # True=展期，False=不展期

# 所有路徑的payoff，折現至t=0
#   在t=6時，不展期的payoff1
#   在t=9時，展期的payoff2、在t=6時支付的展期費 
payoff = payoff1 * ~extFlag * np.exp(r*-t1)  \
        + payoff2 * extFlag * np.exp(r*-(t1+t2))  + extension_fee * extFlag * np.exp(r*t1) 

option_price = np.mean( payoff )
payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print("option_price: ", option_price)
print("CI: ", CI)

option_price:  10.021069839473634
CI:  [ 9.77199534 10.27014434]


# **2. Writer-Extendible Call**
Consider a writer-extendible call on a stock with original time to maturity six months, that will be extended three months if the option is out-of-the-money at original maturity time. The stock price is 80, and the
initial strike price is 90. If the option is extended, the strike price is adjusted to 82. The risk-free interest rate is 10%,
and the volatility is 30%. (6.8238)

In [47]:
S0 = 80
K1 = 90
K2 = 82
r = 0.1
q = 0
sigma = 0.3
t1 = 6/12
t2 = 3/12
nSteps = 900
nPaths = 10000
gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, t1+t2)

payoff1 = np.maximum(gbm.df.loc[t1] - K1, 0)
payoff2 = np.maximum(gbm.df.loc[t1+t2] - K2, 0)
extFlag = gbm.df.loc[t1] < K1   # True=價外會展期

payoff = payoff1 * ~extFlag * np.exp(r*-t1) + payoff2 * extFlag * np.exp(r*-(t1+t2))
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

6.882832185812834
[6.68103835 7.08462602]


# **3. Forward Start Call**
Consider an employee who receives a call option with forward start three months from today. The option starts 10% out-of-the-money, time to maturity is one year from today, the stock price is 60, the risk-free interest rate is 8%, the continuous dividend yield is 4%, and the expected volatility of the stock is 30%. (4.4064)

In [280]:
S0 = 60
r = 0.08
q = 0.04
sigma = 0.3
t1 = 3/12
T = 1
nSteps = 1200
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T)
strike = np.array(gbm.df.loc[t1] * 1.1)

payoff = np.maximum(gbm.St[-1] - strike, 0) * np.exp(-r * (T))
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

4.406956237382712
[4.22793412 4.58597835]


# **4. Asian Put**

Consider an Asian currency put option with a time to expiration of six months. The spot price is 6.80, the strike is 6.90, the domestic risk-free interest rate is 7% per year, the foreign interest rate is 9% per year, and the volatility of the spot rate is 14%. (0.2237)

In [285]:
S0 = 6.8
K = 6.9
r = 0.07
q = 0.09
sigma = 0.14
T = 6/12

nSteps = 1000
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T)
Stmean = gbm.St.mean(axis=0) # 每條路徑上，過去6個月的價格平均

payoff = np.maximum(K - Stmean, 0) * np.exp(-r*T)
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

0.22317286776948003
[0.22161538 0.22473035]


# **5. Chooser Call&Put**
Consider a complex chooser option that gives the holder the right to choose whether the option is to be a
call with six months to expiration and strike price 55, or a put with seven months to expiration and strike price 48. The
time to choose between a put or call is in three months, the underlying stock price is 50, the risk-free interest rate per
year is 10%, the dividend yield is 5% per year, and the volatility per year is 35%. (6.0508)

In [84]:
S0 = 50
K1 = 55
K2 = 48
r = 0.1
q = 0.05
sigma = 0.35
t1 = 3/12
T1 = 6/12
T2 = 7/12

nSteps = 700
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T2)
gbm.df.index = np.round(gbm.df.index, 4)

callprice = call(gbm.df.loc[t1], K1, r, q, sigma, T1-t1).price
putprice = put(gbm.df.loc[t1], K2, r, q, sigma, T2-t1).price
# 在t=3m時，若call>put，則選擇call=True
chooseCall = callprice > putprice

callPayoff = np.maximum(gbm.df.loc[round(T1,4)] - K1, 0)
putPayoff = np.maximum(K2 - gbm.df.loc[round(T2,4)], 0)
payoff = callPayoff * chooseCall * np.exp(-r*T1) + putPayoff * ~chooseCall * np.exp(-r*T2)
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

6.0713875265698745
[5.92060469 6.22217036]


# **6. Lookback Call**
(Lookback) Consider a lookback call option with six months left to expiration. Assume it gives the right to buy the
underlying stock index at the lowest price recorded during the life of the option and that the minimum stock index price
observed so far is 100, the stock price is 120, the risk-free interest rate is 10%, the dividend yield is 6%, and the volatility
is 30%. (25.3533)

In [85]:
S0 = 120
K = 100
r = 0.1
q = 0.06
sigma = 0.3
T = 6/12

nSteps = 1000
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T)
Stmin = gbm.St.min(axis=0) # 每條路徑上，過去6個月的價格最低點
lower = K > Stmin  # 最低點低於100，True=用最低點當履約價

payoff = np.maximum( gbm.St[-1] - ( Stmin * lower + K * ~lower ), 0 ) * np.exp(-r*T) 
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

25.141815060651705
[24.72965813 25.55397199]


# **7. Compound Call**
Consider a put-on-call option that gives the option holder the right to sell a call option for 50, three
months from today. The strike on the underlying call option is 520, the time to maturity on the call is six months from
today, the price on the underlying stock index is 500, the risk-free interest rate is 8%, and the stock index pays dividends
at a rate of 3% annually and has a volatility of 35%. (21.1965)

In [87]:
steps = 100
S0 = 500
K1 = 50      # compound put strike
K2 = 520     # call strike
r = 0.08
q = 0.03
sigma = 0.35
T1 = 3/12
T2 = 6/12

nSteps = 1000
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T1)
callprice = call(gbm.St[-1], K2, r, q, sigma, T2-T1).price # t=3m時，underlying call的價值(剩餘3m到期)
payoff = np.maximum(K1 - callprice, 0) *  np.exp(-r*T1)
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

21.07136708189534
[20.70475785 21.43797632]


# **8. Look-Barrier Call**
Look-barrier options can be regarded as a combination of a partial-time barrier option and a forward starting **fixed-strike lookback option**. Consider a look-barrier call option with one year to expiry. The option’s barrier monitoring period starts at the option’s starting date and ends at six months. If the barrier is not hit the barrier price 120 during the monitoring period, the fixed-strike lookback option will be initiated at the same time the barrier ceases to exist. The stock price and strike price are 100, the risk-free interest rate is 10%, and the expected volatility of the stock is 30%. (7.5509)

In [125]:
S0 = 100
barrier = 120
K = 100
r = 0.1
q = 0
sigma = 0.3
t1 = 0.5
T = 1

nSteps = 1000
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T)
outFlag = (gbm.df.loc[:t1] > barrier).cumsum().loc[t1]  # 超過barrier累積
initiate = (outFlag == 0)  # True=從未超過barrier
Stmax = gbm.df.loc[t1:].max(axis=0)  # 每條路徑上，價格最高點

payoff =  np.maximum( Stmax - K, 0 ) * np.exp(-r*(T)) * initiate  # 沒出局的
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

7.575070130498716
[7.30105413 7.84908613]



# **9. Double-Barrier Binary**
Consider a double-barrier knock-out binary option with a time to expiration of three months.The option pays off zero if the asset price touches the lower 80 or upper 120 barrier before expiration. The option pays
off a cash amount 10 at maturity if the barriers are not hit during the lifetime of the option. The stock price is 100, the
risk-free interest rate is 5%, the continuous dividend yield is 2%, and the expected volatility of the stock is 30%. (6.3272)

In [130]:
S0 = 100
barrier_upper = 120
barrier_lower = 80
cash = 10
r = 0.05
q = 0.02
sigma = 0.3
T = 3/12

nSteps = 1000
nPaths = 10000

gbm = GeometricBrownianMotion(nSteps, nPaths, r-q, S0, sigma, T)
outFlag = ( (gbm.df >= barrier_upper).cumsum() + (gbm.df <= barrier_lower).cumsum()) .loc[T]
payoff = (outFlag == 0) * cash * np.exp(-r*T)
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

6.334324012368539
[6.24148752 6.42716051]



# **10. Rainbow Put**
Consider a put option that gives the holder the right to sell the maximum of stock index A and stock index B at a strike price of 98. Time to maturity is six months, stock index A pays a dividend yield of 6%, stock index B pays a dividend yield of 9%, the price of index A is currently 100, the price of index B is 105, the volatility of index A is 11%, the volatility of index B is 16%, the risk-free interest rate is 5%, and the correlation between the return on the two stock indexes is 0.63. (1.2181)

In [278]:
S0 = np.array([100, 105])
K = np.array([98, 98])
sigma = np.array([0.11, 0.16])
r = 0.05
q = np.array([0.06, 0.09])
rho = 0.63
T = 0.5
nPaths = 100000

Z = np.random.randn(nPaths, 2)
Z[:, 1] = rho * Z[:, 0] + Z[:, 1] * np.sqrt(1 - rho ** 2)
Rt = ((r-q) - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z
lnSt = np.log(S0) + Rt
St = np.exp(lnSt)

payoff = np.maximum(K - St, 0)  # 2個股票指數各自的payoff
payoff = np.minimum(payoff[:, 0], payoff[:, 1]) * np.exp(-r*T)  # 比較低的股票指數的payoff
option_price = payoff.mean()

payoff_sd = np.sqrt( sum((payoff - option_price)**2)/(nPaths-1) ) 
SE = payoff_sd / np.sqrt(nPaths)
CI = option_price + np.array([-1.96, 1.96]) * SE
print(option_price)
print(CI)

1.2106861147554306
[1.19338631 1.22798592]
